In [5]:
import utils

import pandas as pd
import numpy as np

import re
import warnings

pd.set_option('display.max_columns', None)
warnings.filterwarnings('ignore')

df_train = pd.read_csv('/Users/jiayi/Desktop/B_CS671/kaggle/raw_data/train.csv')
df_test = pd.read_csv('/Users/jiayi/Desktop/B_CS671/kaggle/raw_data/test.csv')

"""
Choices for cleaning
-----------------------------------
median       / mice
kick outlier / reserve outlier
category one hot encoded?

"""

def train_clean_1_(df):
    

    median_values = {}
    bound_values = {}

    ## copy
    df_clean = df.copy()

    ## drop col and na row
    df_clean = df_clean.drop(['host_id', 'id', 'scrape_id', 'last_scraped', 'picture_url', 'calendar_last_scraped', 'host_name',
                              'description', 'amenities', 'neighbourhood_cleansed', 'property_type'], axis=1)
    df_clean = df_clean.dropna(subset = ['bathrooms_text'])

    ## encode 5 binary & impute 1var / host_is_superhost & save 
    bin_cols = ['host_is_superhost', 'host_has_profile_pic', 'host_identity_verified', 'has_availability', 'instant_bookable']
    for col in bin_cols:
        df_clean[col] = utils.encode_boolean(df_clean[col])

    super_median, imputed_col = utils.median_impute_numeric(df_clean['host_is_superhost'])
    df_clean['host_is_superhost'] = imputed_col
    median_values['host_is_superhost'] = super_median


    ## convert date
    df_clean['host_since'] = utils.get_days_since_date(df_clean['host_since'])



    ## impute*1: detect outliers
    outlier_cols = ['beds', 'accommodates', 
                    'minimum_nights', 'maximum_nights', 
                    'minimum_minimum_nights', 'maximum_minimum_nights', 'minimum_maximum_nights', 'maximum_maximum_nights',
                    'minimum_nights_avg_ntm', 'maximum_nights_avg_ntm']
    for col in outlier_cols:
        df_clean, lower_bound, upper_bound = utils.outlier_to_na(df_clean, col)
        bound_values[col] = {'lower_bound': lower_bound, 'upper_bound': upper_bound}

    ## impute*2: impute median
    for col in outlier_cols:
        median, df_clean[col] = utils.median_impute_numeric(df_clean[col])
        median_values[col] = median



    ## encode verification
    df_clean = utils.split_and_encode_verification(df_clean)

    ## name convert
    name_data = utils.extract_name_column_info(df_clean)
    df_clean.rename(columns={
        'beds': 'beds_provid'}, inplace=True)
    df_clean = pd.concat((df_clean, name_data), axis = 1)
    df_clean = df_clean.drop(['name', 'bathrooms_text', 'beds_provid'], axis=1)
    


    ## convert to category col
    cat_cols = ['neighbourhood_group_cleansed', 'room_type']
    for col in cat_cols:
        df_clean = utils.one_hot_encoding(df_clean, col)
    df_clean = df_clean.drop(cat_cols, axis=1)
    
    ## impute 4 from name
    name_cols = ['rating', 'beds', 'bedrooms', 'bath_num']
    for col in name_cols:
        median, df_clean[col] = utils.median_impute_numeric(df_clean[col])
        median_values[col] = median
    

    ## change order, price last
    feature_cols = [col for col in df_clean.columns if col != 'price']
    cols_order = feature_cols + ['price']

    # Reorder the DataFrame
    df_clean = df_clean[cols_order]

    return df_clean, median_values, bound_values



## cleaning for test set
def test_clean_1_(df, median_dict, test = True):

    ## copy
    df_clean = df.copy()
    ## drop col and na row
    df_clean = df_clean.drop(['host_id', 'id', 'scrape_id', 'last_scraped', 'picture_url', 'calendar_last_scraped', 'host_name',
                              'description', 'amenities', 'neighbourhood_cleansed', 'property_type'], axis=1)

    ## encode 5 binary & no impute if train = True
    bin_cols = ['host_is_superhost', 'host_has_profile_pic', 'host_identity_verified', 'has_availability', 'instant_bookable']
    for col in bin_cols:
        df_clean[col] = utils.encode_boolean(df_clean[col])
    ## impute for host_is_superhost
    super_median = median_dict['host_is_superhost']
    df_clean['host_is_superhost'] = col.fillna(super_median)


    ## convert date
    df_clean['host_since'] = utils.get_days_since_date(df_clean['host_since'])


    ## impute*1: detect outliers
    outlier_cols = ['beds', 'accommodates', 
                    'minimum_nights', 'maximum_nights', 
                    'minimum_minimum_nights', 'maximum_minimum_nights', 'minimum_maximum_nights', 'maximum_maximum_nights',
                    'minimum_nights_avg_ntm', 'maximum_nights_avg_ntm']
    for col in outlier_cols:
        df_clean, _, _ = utils.outlier_to_na(df_clean, col)

    ## impute*2: impute median
    for col in outlier_cols:
        median = median_dict[col]
        df_clean[col] = col.fillna(median)
   

    ## encode verification
    df_clean = utils.split_and_encode_verification(df_clean)
    

    ## name convert
    name_data = utils.extract_name_column_info(df_clean)
    df_clean.rename(columns={
        'beds': 'beds_provid'}, inplace=True)
    df_clean = pd.concat((df_clean, name_data), axis = 1)
    df_clean = df_clean.drop(['name', 'bathrooms_text', 'beds_provid'], axis=1)
    


    ## convert to category col
    cat_cols = ['neighbourhood_group_cleansed', 'room_type']
    for col in cat_cols:
        df_clean = utils.one_hot_encoding(df_clean, col)
    df_clean = df_clean.drop(cat_cols, axis=1)
    
    ## impute 4 from name
    name_cols = ['rating', 'beds', 'bedrooms', 'bath_num']
    for col in name_cols:
        median = median_dict[col]
        df_clean[col] = col.fillna(median)
    

    if test == False:
        ## change order, price last
        feature_cols = [col for col in df_clean.columns if col != 'price']
        cols_order = feature_cols + ['price']
        # Reorder the DataFrame
        df_clean = df_clean[cols_order]


    return df_clean


In [6]:
df_train_clean, _ = train_clean_1_(df_train)

In [ ]:
print(df_train_clean.shape)
df_train_clean.head(3)

In [ ]:
print('Shape of df_clean: ', df_clean.shape)   ## (15441, 48)
info_df = pd.DataFrame({'DataType': df_clean.dtypes, 
                        'NumOfNA': df_clean.isna().sum(), 
                        '%OfNA': df_clean.isnull().mean()*100})
info_df